In [30]:
#!jupyter nbextension install --py mayavi --user
#!jupyter nbextension enable mayavi --user --py

Enabling notebook extension mayavi/x3d/x3dom...
      - Validating: OK


## This notebook generates the meshes required for the 3D visualization

The meshes are based on the MASSP atlas. A marching cubes algorithm is used to generate the meshes, after which they are decimated to decrease size & improve performance.

Requires meshlab

In [3]:
import nibabel as nib
import glob
import os
import nilearn
from nilearn import plotting
import nighres
import shutil
import re
# import pymeshlab
import numpy

from skimage.measure import marching_cubes

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7fa80adb2dc0>
Traceback (most recent call last):
  File "/home/stevenm/miniconda3/lib/python3.9/site-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/home/stevenm/miniconda3/lib/python3.9/site-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
  File "/home/stevenm/miniconda3/lib/python3.9/site-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
  File "/home/stevenm/miniconda3/lib/python3.9/ctypes/__init__.py", line 382, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: dlopen() error
Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_ca

In [4]:
def write_obj(fn, verts, faces, normals):
    faces += 1  # 1-based
    with open(fn, 'w') as f:
        for item in verts:
            f.write("v {0} {1} {2}\n".format(item[0],item[1],item[2]))

        for item in normals:
            f.write("vn {0} {1} {2}\n".format(item[0],item[1],item[2]))

        for item in faces:
            f.write("f {0}//{0} {1}//{1} {2}//{2}\n".format(item[0],item[1],item[2]))  

        f.close()

def read_obj(fn):
    import re
    faces_regex = re.compile('f (?P<a>\d+)//(?P<b>\d+) (?P<c>\d+)//(?P<d>\d+) (?P<e>\d+)//(?P<f>\d+)\n')
    faces = []
    normals = []
    verts = []
    with open(fn, 'r') as f:
        obj_file = f.readlines()
        for line in obj_file:
            if line[0:2] == 'vn':
                normals.append(np.array([float(x) for x in line.split(' ')[1:]]))
            elif line[0] == 'v':
                verts.append(np.array([float(x) for x in line.split(' ')[1:]]))
            elif line[0] == 'f':
                if '//' in line:
                    gd = faces_regex.match(line)
                    faces.append(np.array([int(gd['a'])-1, int(gd['c'])-1, int(gd['e'])-1]))
                else:
                    faces.append(np.array([float(x) for x in line.split(' ')[1:]])-1)
                
    verts = np.array(verts)
    normals = np.array(normals)
    faces = np.array(faces)
    return verts, faces, normals

## Original masks of MASSP

In [85]:
massp_masks = sorted(glob.glob('/home/stevenm/Projects/trondheim/sourcedata/massp_young_atlas_from_figshare/massp2021-subcortex_avg-*_hem-*_decade-18to40_n54.nii.gz'))
# all_masks

In [ ]:
target_dir = './objs/massp/original_meshes'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

regex = re.compile('.*massp2021-subcortex_avg-(?P<roi>\S+)_hem-(?P<hemisphere>\S+)_.*-18to40_n54.ni*')

all_masks = {}
for mask in massp_masks:
    print(mask)
    roi_name = regex.match(mask).groupdict()['roi']
    hemisphere = regex.match(mask).groupdict()['hemisphere']
    mask_name = regex.match(mask).groupdict()['roi'] + regex.match(mask).groupdict()['hemisphere']
    data = nib.load(mask).get_fdata()
    verts, faces, normals, values = marching_cubes(data)
    write_obj(os.path.join(target_dir, '{}-{}.obj'.format(roi_name, hemisphere)), verts, faces, normals)

In [86]:
# Some cleaning
import pymeshlab

for fn in sorted(glob.glob('./objs/massp/original_meshes/*.obj')):
    print(fn, end='..')
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fn)
#    ms.meshing_repair_non_manifold_vertices()
#    ms.meshing_repair_non_manifold_edges()
    # ms.meshing_decimation_edge_collapse_for_marching_cube_meshes()
    ms.apply_coord_laplacian_smoothing()
    ms.meshing_re_orient_faces_coherently()
#    ms.meshing_invert_face_orientation()
    # ms.meshing_decimation_quadric_edge_collapse(targetperc=.7)

    
#     # simplify here
#     ms.
    
    ms.save_current_mesh('./objs/massp/' + os.path.basename(fn))

./objs/massp/original_meshes/amg-l.obj.../objs/massp/original_meshes/amg-r.obj.../objs/massp/original_meshes/cl-l.obj.../objs/massp/original_meshes/cl-r.obj.../objs/massp/original_meshes/fx-lr.obj.../objs/massp/original_meshes/gpe-l.obj.../objs/massp/original_meshes/gpe-r.obj.../objs/massp/original_meshes/gpi-l.obj.../objs/massp/original_meshes/gpi-r.obj.../objs/massp/original_meshes/ic-l.obj.../objs/massp/original_meshes/ic-r.obj.../objs/massp/original_meshes/pag-l.obj.../objs/massp/original_meshes/pag-r.obj.../objs/massp/original_meshes/ppn-l.obj.../objs/massp/original_meshes/ppn-r.obj.../objs/massp/original_meshes/rn-l.obj.../objs/massp/original_meshes/rn-r.obj.../objs/massp/original_meshes/sn-l.obj.../objs/massp/original_meshes/sn-r.obj.../objs/massp/original_meshes/stn-l.obj.../objs/massp/original_meshes/stn-r.obj.../objs/massp/original_meshes/str-l.obj.../objs/massp/original_meshes/str-r.obj.../objs/massp/original_meshes/tha-l.obj.../objs/massp/original_meshes/tha-r.obj.../objs/m

## Original masks of Str (MASSP 2)

In [5]:
str_masks = sorted(glob.glob('./sourcedata/massp2/proba_ahead-massp2_avg-cau_hem-*_decade-18to40_n50.nii.gz')) + \
sorted(glob.glob('./sourcedata/massp2/proba_ahead-massp2_avg-acc_hem-*_decade-18to40_n50.nii.gz')) + \
sorted(glob.glob('./sourcedata/massp2/proba_ahead-massp2_avg-put_hem-*_decade-18to40_n50.nii.gz'))
# str_masks

In [6]:
target_dir = './objs/massp2/original_meshes'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

regex = re.compile('.*proba_ahead-massp2_avg-(?P<roi>\S+)_hem-(?P<hemisphere>\S+)_.*-18to40_n50.ni*')

all_masks = {}
for mask in str_masks:
    print(mask)
    roi_name = regex.match(mask).groupdict()['roi']
    hemisphere = regex.match(mask).groupdict()['hemisphere']
    mask_name = regex.match(mask).groupdict()['roi'] + regex.match(mask).groupdict()['hemisphere']
    data = nib.load(mask).get_fdata()
    verts, faces, normals, values = marching_cubes(data)
    write_obj(os.path.join(target_dir, '{}-{}.obj'.format(roi_name, hemisphere)), verts, faces, normals)

./sourcedata/massp2/proba_ahead-massp2_avg-cau_hem-l_decade-18to40_n50.nii.gz
./sourcedata/massp2/proba_ahead-massp2_avg-cau_hem-r_decade-18to40_n50.nii.gz
./sourcedata/massp2/proba_ahead-massp2_avg-acc_hem-l_decade-18to40_n50.nii.gz
./sourcedata/massp2/proba_ahead-massp2_avg-acc_hem-r_decade-18to40_n50.nii.gz
./sourcedata/massp2/proba_ahead-massp2_avg-put_hem-l_decade-18to40_n50.nii.gz
./sourcedata/massp2/proba_ahead-massp2_avg-put_hem-r_decade-18to40_n50.nii.gz


In [94]:
import pymeshlab

# let's use the young subjects
for fn in sorted(glob.glob('./objs/massp2/original_meshes/*.obj')):
    print(fn, end='..')
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fn)
#    ms.meshing_repair_non_manifold_vertices()
#    ms.meshing_repair_non_manifold_edges()
#    ms.meshing_decimation_edge_collapse_for_marching_cube_meshes()
    ms.apply_coord_laplacian_smoothing()
    ms.meshing_re_orient_faces_coherently()
#    ms.meshing_invert_face_orientation()
    # ms.meshing_decimation_quadric_edge_collapse(targetperc=.7)

    
#     # simplify here
#     ms.
    
    ms.save_current_mesh('./objs/massp2/' + os.path.basename(fn))

./objs/massp2/original_meshes/acc-l.obj.../objs/massp2/original_meshes/acc-r.obj.../objs/massp2/original_meshes/cau-l.obj.../objs/massp2/original_meshes/cau-r.obj.../objs/massp2/original_meshes/put-l.obj.../objs/massp2/original_meshes/put-r.obj..

## Original masks of Thalamus

In [58]:
# mask = '/home/Public/trondheim/derivatives/thalamic_segmentations/0p5mm/sub-002/sub-002_roi-LGN-l_res-0p5.nii.gz'

# regex_masks = re.compile('.*_roi-(?P<label>\S+)-(?P<hemisphere>\S)_res-0p5.nii.gz')
# regex_masks.match(mask).groupdict()
# t1w = '/home/Public/trondheim/derivatives/fmriprep/fmriprep/sub-002/anat/sub-002_desc-preproc_T1w.nii.gz'
# plotting.plot_stat_map(mask, bg_img=t1w)

{'label': 'LGN', 'hemisphere': 'l'}

In [64]:
all_subs = sorted(glob.glob('/home/Public/trondheim/derivatives/thalamic_segmentations/0p5mm/*'))
all_subs = [x.split('-')[-1] for x in all_subs]

In [66]:
## Warp individual subjects' parcellations to MNI-space
import multiprocess as mp
from utils import *
def warp_masks_2_mni(sub, overwrite=False):
    print(sub)
#     regex_masks = re.compile('.*/space-MNI152NLin2009cAsym_label-(?P<label>\S+)_probseg.nii.gz')
    regex_masks = re.compile('.*_roi-(?P<label>\S+)-(?P<hemisphere>\S)_res-0p5.nii.gz')
    all_masks = sorted(glob.glob(f'/home/Public/trondheim/derivatives/thalamic_segmentations/0p5mm/sub-{sub}/sub-{sub}_roi-*-*_res-0p5.nii.gz'))
    
    output_img_base = f'./thalamus_mni/sub-{sub}/anat/sub-{sub}_space-MNI152NLin2009cAsym_desc-mask-{{label}}-{{hemisphere}}.nii.gz'
    
    for mask in all_masks:
        mask_label = regex_masks.match(mask).groupdict()['label']
        mask_hemisphere = regex_masks.match(mask).groupdict()['hemisphere']
        
        final_img_fn = output_img_base.format(label=mask_label, hemisphere=mask_hemisphere)
        if not os.path.exists(final_img_fn) or overwrite:
            os.makedirs(os.path.dirname(final_img_fn), exist_ok=True)
            tmp_img_name = apply_warp(mask, sub=sub, t1w_to_MNI=True)
            shutil.move(tmp_img_name, final_img_fn)

with mp.Pool(20) as p:
    p.map(warp_masks_2_mni, all_subs)

003002004

005008006007
009




010
011012013

014
015

018020019021



016017

022
023
024
025
026
027
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044


In [87]:
## Get mean across subjects
all_mask_names = sorted(glob.glob('/home/Public/trondheim/derivatives/thalamic_segmentations/0p5mm/sub-002/sub-002_roi-*_res-0p5.nii.gz'))
regex_masks = re.compile('.*_roi-(?P<label>\S+)-(?P<hemisphere>\S)_res-0p5.nii.gz')
all_mask_names = [regex_masks.match(x).groupdict() for x in all_mask_names]
for mask_dict in all_mask_names:
    label, hemisphere = mask_dict['label'], mask_dict['hemisphere']
    output_fn = f'thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-{label}-{hemisphere}_avg.nii.gz'
    if os.path.exists(output_fn):
        continue
    fns = sorted(glob.glob(f'thalamus_mni/sub-*/anat/sub-*_space-MNI152NLin2009cAsym_desc-mask-{label}-{hemisphere}.nii.gz'))
    mn = nilearn.image.mean_img(fns)
    mn.to_filename(output_fn)

## Average fns now created, next up: Make meshes

/tmp/ipykernel_552535/415347458.py:11: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



In [92]:
#thal_masks = sorted(glob.glob('/home/Public/trondheim/derivatives/thalamic_segmentations/mni09c/mni09c_roi-*-*_res-0p5.nii.gz'))
thal_masks = sorted(glob.glob('thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-*-*_avg.nii.gz'))
thal_masks

target_dir = './objs/thal/original_meshes'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

regex = re.compile('.*space-MNI152NLin2009cAsym_desc-mask-(?P<roi>\S+)-(?P<hemisphere>\S+)_avg.nii.gz')
# regex = re.compile('.*mni09c_roi-(?P<roi>\S+)-(?P<hemisphere>\S+)_res-0p5.ni*')
#regex.match(thal_masks[20]).groupdict()

all_masks = {}
for mask in thal_masks:
    print(mask)
    if 'PaV' in mask or 'Pc' in mask or 'Pt' in mask or '-R-' in mask or '-VM-' in mask: # too small
        continue
    roi_name = regex.match(mask).groupdict()['roi']
    hemisphere = regex.match(mask).groupdict()['hemisphere']
    mask_name = regex.match(mask).groupdict()['roi'] + regex.match(mask).groupdict()['hemisphere']
    ## reorient for consistent orientaiton of meshes
    data = nilearn.image.resample_to_img(mask, target_img=massp_masks[0]).get_fdata()
#    data = nib.load(mask).get_fdata()
    verts, faces, normals, values = marching_cubes(data)
    write_obj(os.path.join(target_dir, '{}-{}.obj'.format(roi_name, hemisphere)), verts, faces, normals)

thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-AV-l_avg.nii.gz


/tmp/ipykernel_552535/2424108890.py:22: FutureWarning:

'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.

/home/stevenm/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:807: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-AV-r_avg.nii.gz


/tmp/ipykernel_552535/2424108890.py:22: FutureWarning:

'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.

/home/stevenm/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:807: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-CL-l_avg.nii.gz


...

/tmp/ipykernel_552535/2424108890.py:22: FutureWarning:

'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.

/home/stevenm/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:807: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-VM-l_avg.nii.gz
thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-VM-r_avg.nii.gz
thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-VPL-l_avg.nii.gz


/tmp/ipykernel_552535/2424108890.py:22: FutureWarning:

'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.

/home/stevenm/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:807: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



thalamus_mni/space-MNI152NLin2009cAsym_desc-mask-VPL-r_avg.nii.gz


/tmp/ipykernel_552535/2424108890.py:22: FutureWarning:

'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.

/home/stevenm/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:807: FutureWarning:

From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.



In [95]:
import pymeshlab

# let's use the young subjects
for fn in sorted(glob.glob('./objs/thal/original_meshes/*.obj')):
    print(fn, end='..')
    if '/CL-' in fn:
        continue
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fn)
#    ms.meshing_repair_non_manifold_vertices()
#    ms.meshing_repair_non_manifold_edges()
#    ms.meshing_decimation_edge_collapse_for_marching_cube_meshes()
    ms.apply_coord_laplacian_smoothing()
    ms.meshing_re_orient_faces_coherently()
#    ms.meshing_invert_face_orientation()
    # ms.meshing_decimation_quadric_edge_collapse(targetperc=.7)

    
#     # simplify here
#     ms.
    
    ms.save_current_mesh('./objs/thal/' + os.path.basename(fn))

./objs/thal/original_meshes/AV-l.obj.../objs/thal/original_meshes/AV-r.obj.../objs/thal/original_meshes/CL-l.obj.../objs/thal/original_meshes/CL-r.obj.../objs/thal/original_meshes/CM-l.obj.../objs/thal/original_meshes/CM-r.obj.../objs/thal/original_meshes/CeM-l.obj.../objs/thal/original_meshes/CeM-r.obj.../objs/thal/original_meshes/LD-l.obj.../objs/thal/original_meshes/LD-r.obj.../objs/thal/original_meshes/LGN-l.obj.../objs/thal/original_meshes/LGN-r.obj.../objs/thal/original_meshes/LP-l.obj.../objs/thal/original_meshes/LP-r.obj.../objs/thal/original_meshes/LSg-l.obj.../objs/thal/original_meshes/LSg-r.obj.../objs/thal/original_meshes/MDl-l.obj.../objs/thal/original_meshes/MDl-r.obj.../objs/thal/original_meshes/MDm-l.obj.../objs/thal/original_meshes/MDm-r.obj.../objs/thal/original_meshes/MGN-l.obj.../objs/thal/original_meshes/MGN-r.obj.../objs/thal/original_meshes/MV-l.obj.../objs/thal/original_meshes/MV-r.obj.../objs/thal/original_meshes/Pf-l.obj.../objs/thal/original_meshes/Pf-r.obj..

In [7]:
# pymeshlab.pmeshlab.MeshSet.meshing_invert_face_orientation()

TypeError: filter_function() missing 1 required positional argument: 'self'

In [8]:
# thal_masks = 

In [ ]:
# from mayavi import mlab
# mlab.init_notebook()

# fig = mlab.figure(bgcolor=(0,0,0))
# s = mlab.test_plot3d()
# s

## Simplify meshes
Makes use of meshlab's decimate

In [6]:
import meshlabxml

ModuleNotFoundError: No module named 'meshlabxml'

In [14]:
def simplify_mesh(in_file, out_file, target_perc=0.01):
    import meshlabxml as mlx
    import os
    script = 'simplify.mlx' # script file
    
    mlx.begin(script=script, file_in=in_file) # Start writing the script to the script file
    mlx.remesh.simplify(script=script, texture=False, faces=1000,
                        target_perc=target_perc, quality_thr=0.3, preserve_boundary=False,
                        boundary_weight=1.0, preserve_normal=False,
                        optimal_placement=True, planar_quadric=False,
                        selected=False, extra_tex_coord_weight=1.0,
                        preserve_topology=True, quality_weight=False,
                        autoclean=True)
    mlx.end(script=script) # Finish writing the script to the script file
    
    call = ['xvfb-run', '-a', '-s', '"-screen 0 800x600x24"', "meshlabserver", "-i", 
                     in_file, "-o", 
                     out_file, "-s", "simplify.mlx"]
    os.system(' '.join(call))

In [18]:
# all_objs = sorted(glob.glob('app/data/ahead_objs/original_meshes/*.obj'))
# all_objs

# for target_perc in [0.01, 0.05, 0.1]:
#     save_dir = os.path.dirname(all_objs[0]).replace('original_meshes', 'simplified-{}'.format(str(target_perc).replace('.', 'p')))
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)
    
#     for obj in all_objs:
#         print(obj)
#         simplify_mesh(in_file=obj, out_file=obj.replace('original_meshes', 'simplified-{}'.format(str(target_perc).replace('.', 'p'))),
#                       target_perc=target_perc)

app/data/ahead_objs/original_meshes/amg-l.obj
app/data/ahead_objs/original_meshes/amg-r.obj
app/data/ahead_objs/original_meshes/cl-l.obj
app/data/ahead_objs/original_meshes/cl-r.obj
app/data/ahead_objs/original_meshes/fx-lr.obj
app/data/ahead_objs/original_meshes/gpe-l.obj
app/data/ahead_objs/original_meshes/gpe-r.obj
app/data/ahead_objs/original_meshes/gpi-l.obj
app/data/ahead_objs/original_meshes/gpi-r.obj
app/data/ahead_objs/original_meshes/ic-l.obj
app/data/ahead_objs/original_meshes/ic-r.obj
app/data/ahead_objs/original_meshes/pag-l.obj
app/data/ahead_objs/original_meshes/pag-r.obj
app/data/ahead_objs/original_meshes/ppn-l.obj
app/data/ahead_objs/original_meshes/ppn-r.obj
app/data/ahead_objs/original_meshes/rn-l.obj
app/data/ahead_objs/original_meshes/rn-r.obj
app/data/ahead_objs/original_meshes/sn-l.obj
app/data/ahead_objs/original_meshes/sn-r.obj
app/data/ahead_objs/original_meshes/stn-l.obj
app/data/ahead_objs/original_meshes/stn-r.obj
app/data/ahead_objs/original_meshes/str-l.

## Use meshes created for figures

/home/stevenm/Projects/Ahead_statistics/code/app


In [1]:
import pymeshlab

Unable to load the following plugins:

	libio_e57.so: libio_e57.so does not seem to be a Qt Plugin.

Cannot load library /home/stevenm/miniconda3/lib/python3.9/site-packages/pymeshlab/lib/plugins/libio_e57.so: (/lib/x86_64-linux-gnu/libp11-kit.so.0: undefined symbol: ffi_type_pointer, version LIBFFI_BASE_7.0)



In [7]:
# let's use the young subjects
for fn in sorted(glob.glob('../objs/decade-18to30/bestlabel/*.obj')):
    print(fn, end='..')
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fn)
#     ms.laplacian_smooth()
#     ms.re_orient_all_faces_coherentely()
    ms.invert_faces_orientation()
    
#     ms.simplification_edge_collapse_for_marching_cube_meshes()
#     # simplify here
#     ms.
    ms.simplification_quadric_edge_collapse_decimation(targetperc=0.015)
    
    ms.save_current_mesh('subcortex-app/data/ahead_objs/simplified-0p015/' + os.path.basename(fn))

../objs/decade-18to30/bestlabel/3V-3.obj..../objs/decade-18to30/bestlabel/4V-4.obj..../objs/decade-18to30/bestlabel/AMG-L.obj..../objs/decade-18to30/bestlabel/AMG-R.obj..../objs/decade-18to30/bestlabel/CL-L.obj..../objs/decade-18to30/bestlabel/CL-R.obj..../objs/decade-18to30/bestlabel/GPe-L.obj..../objs/decade-18to30/bestlabel/GPe-R.obj..../objs/decade-18to30/bestlabel/GPi-L.obj..../objs/decade-18to30/bestlabel/GPi-R.obj..../objs/decade-18to30/bestlabel/LV-L.obj..../objs/decade-18to30/bestlabel/LV-R.obj..../objs/decade-18to30/bestlabel/PAG-L.obj..../objs/decade-18to30/bestlabel/PAG-R.obj..../objs/decade-18to30/bestlabel/PPN-L.obj..../objs/decade-18to30/bestlabel/PPN-R.obj..../objs/decade-18to30/bestlabel/RN-L.obj..../objs/decade-18to30/bestlabel/RN-R.obj..../objs/decade-18to30/bestlabel/SN-L.obj..../objs/decade-18to30/bestlabel/SN-R.obj..../objs/decade-18to30/bestlabel/STN-L.obj..../objs/decade-18to30/bestlabel/STN-R.obj..../objs/decade-18to30/bestlabel/STR-L.obj..../objs/decade-18to30

In [ ]:
## Try to optimize? maybe that'll look a bit better